In [1]:
import requests
from bs4 import BeautifulSoup

def get_job_links_page(page):
    base_url = "https://www.indeed.com/jobs?"
    params = {'q': 'data scientist',
             'l': 'Denver, CO'}

    # start = "https://www.indeed.com/jobs?q=Data+Scientist&l=Denver%2C+CO"
    # use a fake header
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}
    params['start'] = 10 * (page-1)

    page = requests.get(base_url, params=params, headers=headers)
    # test = requests.get(start, headers=headers)
    
    soup = BeautifulSoup(page.text, "html.parser")
    links = soup.find_all("a")
    
    # build a list of links
    some_links = []

    for l in links:
        try:
            hyperlink = l.attrs.get('href')
            if "/rc/clk?" in hyperlink:
                some_links.append(l.attrs.get('href'))
        except:
            pass
    
    job_links = ["https://www.indeed.com{}".format(x)
             for x in some_links
             ]
    
    return job_links

In [2]:
def get_job_summary(link):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}
    test = requests.get(link, headers=headers)
    soup = BeautifulSoup(test.text, "html.parser")
    spans = soup.find_all('span')
    spans_w_divs = [span.find_all('div') for span in spans if len(span.find_all('div')) > 0]
    span = soup.find("span", id="job_summary")
    return str(span)

In [3]:
import csv
from nltk.tokenize import RegexpTokenizer

def initialize_highlighting(filename):
# Read a list of skill phrases from a file
    skill_phrases = []
    with open(filename, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            skill_phrases.append(str(*row).strip())
    skill_phrases = set(skill_phrases)

    # Write this clean version back out
    with open('skill_phrases_out.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for skill_phrase in skill_phrases:
            csv_writer.writerow([skill_phrase])
        
    #separate each skill phrase into a list of its words
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+-]+')
    skill_phrase_wl = [tokenizer.tokenize(skill_phrase) for skill_phrase in skill_phrases]
    
    return skill_phrase_wl

In [4]:
def tally_skill_mentions_in_job(t, skill_phrase_wl):
    
    skill_mentions_in_job = defaultdict(int)
    # tokenize the text of the description, without spans
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+-]+')
    tokens = tokenizer.tokenize(t)
    # create a dictionary of the words in the job description
    word_index = defaultdict(list)
    for i, k in enumerate(tokens):
        word_index[k].append(i)
    
    # search the word_index dictionary to find the fist word of each skill_phrase
    for skill_phrase in skill_phrase_wl:
        if word_index.get(skill_phrase[0]):
            for occurence in word_index.get(skill_phrase[0]):
                # Check to see if the whole phrase matches
                if all((skill_phrase[j] == tokens[j+occurence]) for j in range(len(skill_phrase))):
                    skill_mentions_in_job[tuple(skill_phrase)] += 1
    return skill_mentions_in_job        

In [5]:
from collections import defaultdict
import pandas as pd
from IPython.core.display import HTML

# requires the global variable skill_phrase_wl
# maybe this should be a parameter rather than a global
def highlight_phrases_from_list(t):

    # tokenize the text of the description, with spans
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+-]+')
    span_generator = tokenizer.span_tokenize(t)
    spans = [span for span in span_generator]
    tokens = [t[span[0]:span[1]] for span in spans]
    
#     # create a dictionary of the words, with spans as the values
#     # and another dictionary with the same keys, with the word indexes as the values
    char_span = defaultdict(list)
    word_index = defaultdict(list)
    for i, (k, span) in enumerate(zip(tokens, spans)):
        char_span[k].append(span)
        word_index[k].append(i)
    # is this useful?
    df = pd.DataFrame({'Character Index Spans': pd.Series(char_span), 'Word Indexes': pd.Series(word_index)})

    highlight_spans = []   
    for skill_phrase in skill_phrase_wl:
        if word_index.get(skill_phrase[0]):
            for i, occurence in enumerate(word_index.get(skill_phrase[0])):
                if all((skill_phrase[j] == tokens[j+occurence]) for j in range(len(skill_phrase))):
                    highlight_span = (spans[occurence][0], spans[occurence + len(skill_phrase) - 1][1])
                    highlight_spans.append (highlight_span)

# # look up the words in our skill list in the dictionary.  List the findings as spans to be highlighted
#     for skill in single_word_skills:
#         highlight_spans += char_span[skill]

    # Sort the spans to be highlighted
    highlight_spans.sort()

    # Insert html tags to highlight the keywords
    html_start_tag = '<font color="red">'
    html_end_tag = '</font>'
    highlighted = ''
    cursor = 0
    for span in highlight_spans:
        if (span[0] > cursor): # go forwards only, not backwards 
            if (cursor>0):
                highlighted += html_end_tag
            highlighted += t[cursor:span[0]] + \
                            html_start_tag + \
                            t[span[0]:span[1]]
        elif (span[1] > cursor):
            highlighted += t[cursor:span[1]]
        cursor = span[1]
    highlighted += html_end_tag + t[cursor:]
    display(HTML(highlighted))    

In [6]:
job_links = []
for page in range(1,25):
    job_links+=(get_job_links_page(page))

In [7]:
from tqdm import tqdm

tallied_skill_mentions = []
skill_phrase_wl = initialize_highlighting('skill_phrases_purged-JBM.csv')
skill_dict = {tuple(skill_phrase): 0 for skill_phrase in skill_phrase_wl}
for link in tqdm(job_links):
    # highlight_phrases_from_list(get_job_summary(link))
    tallied_skill_mentions.append(tally_skill_mentions_in_job(get_job_summary(link), skill_dict))

100%|████████████████████████████████████████████████████████████████████████████████| 223/223 [02:24<00:00,  1.54it/s]


In [8]:
df = pd.DataFrame(tallied_skill_mentions, index = job_links).fillna(0).astype(int)

In [9]:
skills_mentioned = df.columns.values
skill_phrases_mentioned = [' '.join(c) for c in df.columns.values]
df.columns = skill_phrases_mentioned

In [10]:
df

,A B,AI,APIs,AWS,Access,Adobe,Agile,Android,AngularJS,Apache,...,targeting,templates,test-driven development,time management,unit testing,visualization,web,web application,web development,workflow
https://www.indeed.com/rc/clk?jk=525677fafe696fea&fccid=3c371b712ace44cf&vjs=3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.indeed.com/rc/clk?jk=82d5f1b9af90d82f&fccid=0a0a2172bfedd7b7&vjs=3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
https://www.indeed.com/rc/clk?jk=da4412e0f36920a0&fccid=de71a49b535e21cb&vjs=3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
https://www.indeed.com/rc/clk?jk=c9d84c941c6b6d2f&fccid=4600afcfb0f58ad6&vjs=3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
https://www.indeed.com/rc/clk?jk=d1c86323ed3feec8&fccid=3c371b712ace44cf&vjs=3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.indeed.com/rc/clk?jk=1fce0c6ca709f5ef&fccid=3a13e586bee51cc7&vjs=3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.indeed.com/rc/clk?jk=128ca9d1ab4137b0&fccid=49bacaf9ea5daa22&vjs=3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.indeed.com/rc/clk?jk=3d42198e8b2365bc&fccid=ac1009e3b2f755f3&vjs=3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.indeed.com/rc/clk?jk=3b7429025e95323f&fccid=3c74eafe288fc8ca&vjs=3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
https://www.indeed.com/rc/clk?jk=f3ec178c7a762e5a&fccid=d512f2c02718adce&vjs=3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
not_mentioned = skill_dict
for skill in skills_mentioned:
    del not_mentioned[skill]

In [12]:
phrases_not_mentioned = [' '.join(s) for s in not_mentioned]
phrases_not_mentioned

['React Native',
 'user interaction patterns',
 'command line',
 'Front end web design',
 'database architecture',
 'CRM customer relationship managment',
 'AppEngine',
 'Object-oriented programming',
 'BI Business Intelligence',
 'database administration',
 'speed optimization',
 'web platforms',
 'dispatching',
 'image video rendering',
 'TauCharts',
 'infographics',
 'digital design',
 'iOS engineer',
 'content management workflow',
 'Applications Manager',
 'threads',
 'Angular',
 'MS office',
 'WordPress',
 'Omnigraffle',
 'Instagram',
 'document databases',
 'lead generation',
 'technology consultant',
 'Django python framework',
 'Master degree',
 'Web Developers',
 'web production',
 'Website Management',
 'cross-browser',
 'SASS',
 'CMS content management system',
 'Sketch',
 'JSDoc',
 'AR',
 'web analytics',
 'digital channels',
 'Interactive Design',
 'UX Pin',
 'experimental biology',
 'vendor management',
 'Lucene Solr',
 'Principle',
 'user interface design',
 'Sitecore',

In [13]:
len (phrases_not_mentioned)

131

In [14]:
df.sum(axis = 1).sort_values()

https://www.indeed.com/rc/clk?jk=646b1644cf6811e9&fccid=82c5e27e77990fa2&vjs=3     0
https://www.indeed.com/rc/clk?jk=5b315e8ae866b8d3&fccid=b4ae64835618776c&vjs=3     0
https://www.indeed.com/rc/clk?jk=5738fb6cbf5a1820&fccid=82c5e27e77990fa2&vjs=3     0
https://www.indeed.com/rc/clk?jk=6ea0e326decdf67b&fccid=2e1276ca32b1a903&vjs=3     0
https://www.indeed.com/rc/clk?jk=a416eb0595e1d138&fccid=82c5e27e77990fa2&vjs=3     0
https://www.indeed.com/rc/clk?jk=3ba4964e18046d73&fccid=d9dd442e400888fb&vjs=3     0
https://www.indeed.com/rc/clk?jk=d1efea8374310bf1&fccid=549fdb82108e32a3&vjs=3     0
https://www.indeed.com/rc/clk?jk=9093d1626dddf3a5&fccid=82c5e27e77990fa2&vjs=3     1
https://www.indeed.com/rc/clk?jk=f6ca112d43deb64d&fccid=946656ba950b3b89&vjs=3     1
https://www.indeed.com/rc/clk?jk=04545d34534d4583&fccid=e370394a608927d2&vjs=3     1
https://www.indeed.com/rc/clk?jk=4a117437d30a4e50&fccid=e370394a608927d2&vjs=3     1
https://www.indeed.com/rc/clk?jk=65ddae4b9a75c9fe&fccid=82c5e27e7

In [15]:
skill_phrases = []
with open('skill_phrases_purged-JBM.csv', 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        skill_phrases.append(str(row[]).strip())

In [17]:
skill_phrases=list(set(skill_phrases))

In [33]:
import random
from itertools import compress
skill_percentage = random.random()
n_skill_phrases = len(skill_phrases)
n_my_skills = skill_percentage * n_skill_phrases
my_skills_bool = [(x < n_my_skills) for x in range(n_skill_phrases)]
random.shuffle(my_skills_bool)
my_skill_list = list(compress(skill_phrases,my_skills_bool))
my_skill_list

['C++',
 'React Native',
 'user interaction patterns',
 'Django',
 'managing projects',
 'CRM(customer relationship managment)',
 'Adobe',
 'BI(Business Intelligence)',
 'database administration',
 'XML',
 'communication skills',
 'RESTful',
 'web platforms',
 'unit testing',
 'dispatching',
 'image/video rendering',
 'TauCharts',
 'Applications Manager',
 'threads',
 'Angular',
 'MongoDB',
 'WordPress',
 'Instagram',
 'iOS',
 'document databases',
 'GraphQL',
 'computer programming',
 'lead generation',
 'technology consultant',
 'Django python framework',
 'Master degree',
 'UI/UX',
 'Java',
 'CSS3',
 'programming',
 'server',
 'APIs',
 'sensitive information',
 'data visualization',
 'Linux',
 'MySQL',
 'backend',
 'hardware',
 'web analytics',
 'JavaScript',
 'Interactive Design',
 'UX Pin',
 'UI',
 'shell',
 'Google Cloud',
 'user interface design',
 'SaaS',
 'software',
 'design research',
 'VFX',
 'RESTful APIs',
 'Adobe Creative Suite',
 'hierarchy',
 'scoping projects',
 'clou